In [91]:
import keras.layers as layers
from keras.models import Sequential, Model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from simpler_utils import load_data
import h5py
from tqdm import tqdm

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [41]:
meta_path = './meta.csv'
features_path = './data/features.h5'

X_train, y_train, true_labels, modac, data_size = load_data(features_path, meta_path, 'Train')
modac = np.array(modac)
true_labels = np.array(true_labels)

In [8]:
print(f'Modac has {len(modac)} elements')
print(f'Modac elements are of type {type(modac[0])}')
print(f'Modac elements look like "{modac[0]}"')
print(f'Are X_train and modac the same size? {len(X_train) == len(modac)}\n')

print(f'True labels has {len(true_labels)} elements')
print(f'True labels elements are of type {type(true_labels[0])}')
print(f'True labels elements look like "{true_labels[0]}"')
print(f'Are X_train and true_labels the same size? {len(X_train) == len(true_labels)}\n')

Modac has 139620 elements
Modac elements are of type <class 'str'>
Modac elements look like "a"
Are X_train and modac the same size? True

True labels has 139620 elements
True labels elements are of type <class 'str'>
True labels elements look like "airport"
Are X_train and true_labels the same size? True



In [5]:
scene_labels = np.unique(true_labels)
print(f'There are {len(scene_labels)} scene labels')
print(f'The scene labels are {scene_labels}')

There are 10 scene labels
The scene labels are ['airport' 'bus' 'metro' 'metro_station' 'park' 'public_square'
 'shopping_mall' 'street_pedestrian' 'street_traffic' 'tram']


In [7]:
modac_labels = np.unique(modac)
print(f'There are {len(modac_labels)} modac labels')
print(f'Modac labels are {modac_labels}')

There are 6 modac labels
Modac labels are ['a' 'b' 'c' 's1' 's2' 's3']


In [10]:
model = tf.keras.models.load_model('data/base_model/model.tflite')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 51, 16)        800       
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 51, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 40, 51, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 51, 16)        12560     
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 51, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 40, 51, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 10, 16)         0

In [12]:
pred_1 = model.predict(X_train[0:1])
print(f'Prediction for first element is "{scene_labels[np.argmax(pred_1)]}"')
print(f'True label was "{true_labels[0]}"')

Prediction for first element is "shopping_mall"
True label was "airport"


In [13]:
y_pred = model.predict(X_train)

In [14]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score

In [22]:
def computeMetrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, average='weighted'),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'confusion_matrix': confusion_matrix(y_true, y_pred, normalize='true')
    }

In [23]:
overall_metrics = computeMetrics(true_labels, np.array([scene_labels[np.argmax(pred)] for pred in y_pred]))

In [36]:
for key, value in overall_metrics.items():
    if key == 'confusion_matrix':
        displayed_values = np.array2string(value, formatter={'float_kind':lambda x: "%.2f" % x})
        print(f'\n{key}:\n{displayed_values}')
    else:
        # print it with 2 decimal places and with values aligned to the right
        print(f'{key:<15}: {100*value:.2f} %')

accuracy       : 71.50 %
f1             : 71.59 %
precision      : 72.09 %
recall         : 71.50 %

confusion_matrix:
[[0.72 0.00 0.00 0.03 0.00 0.03 0.08 0.12 0.00 0.00]
 [0.00 0.69 0.05 0.02 0.01 0.03 0.00 0.01 0.02 0.18]
 [0.00 0.05 0.66 0.07 0.01 0.01 0.00 0.01 0.02 0.16]
 [0.06 0.02 0.10 0.63 0.01 0.02 0.04 0.06 0.03 0.04]
 [0.00 0.00 0.00 0.00 0.91 0.03 0.00 0.01 0.03 0.01]
 [0.04 0.01 0.01 0.02 0.02 0.65 0.02 0.17 0.05 0.01]
 [0.14 0.00 0.00 0.05 0.00 0.03 0.69 0.08 0.00 0.00]
 [0.11 0.01 0.00 0.02 0.01 0.20 0.04 0.60 0.01 0.01]
 [0.00 0.01 0.01 0.02 0.05 0.07 0.00 0.01 0.83 0.00]
 [0.01 0.07 0.08 0.03 0.02 0.01 0.00 0.02 0.01 0.76]]


In [42]:
modac_metrics = {}

for label in modac_labels:
    modac_metrics[label] = computeMetrics(true_labels[modac == label], np.array([scene_labels[np.argmax(pred)] for pred in y_pred[modac == label]]))

In [45]:
for label in modac_labels:
    print(f'{label:<5}: {100*modac_metrics[label]["accuracy"]:.2f} %')

a    : 75.37 %
b    : 56.14 %
c    : 63.73 %
s1   : 59.28 %
s2   : 63.77 %
s3   : 61.73 %


On veut ajouter le terme de régularisation suivant dans l'optimisation du modèle, si on décompose le réseau classifieur $\Phi$ en deux parties, l'une qui s'arrête à la fin de la couche 2 de convolution et le reste, respectivement noté $\Phi_1$ et $\Phi_2$, tels que $\Phi=\Phi_2\circ\Phi_1$, on peut écrire le terme de régularisation comme suit:

$$
    \mathcal{L}_{\text{reg}}(X, y, m) := \sum_{y_i = y_j \atop m_i\neq m_j}\|\Phi_1(x_i) - \Phi_1(x_j)\|_1
$$

Pour commencer on va calculer, avec le réseau déjà entrainé avec l'objectif de classification uniquement, des métriques. Notamment on va calculer le terme de régularisation pour le jeu de données d'entraînement entier.

In [78]:
model_1 = Model(
    inputs = model.input,
    outputs = model.get_layer('conv2d_1').output
)
model_1.compile()

In [80]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 40, 51, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 40, 51, 16)        800       
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 51, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 40, 51, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 51, 16)        12560     
Total params: 13,424
Trainable params: 13,392
Non-trainable params: 32
_________________________________________________________________


In [92]:
reg_term_classif_trained = 0

for i, x in tqdm(enumerate(X_train)):
    x_loi = model_1.predict(np.array(tf.expand_dims(x, 0)))[0]
    for j, y in enumerate(X_train):
        if j > i:
            if true_labels[i] == true_labels[j]:
                if modac[i] != modac[j]:
                    y_loi = model_1.predict(np.array(tf.expand_dims(y, 0)))[0]
                    reg_term_classif_trained += np.sum(np.abs(x_loi - y_loi))

17it [32:43, 112.75s/it]